
#  INT303 Big Data Analytics


##  Lab 3 Scraping

**XJTLU**<br>
**S1 2021**<br>
**Instructors:** Jia WANG <br>
**Lab Instructor:** Jia WANG <br>
**Authors:** Rahul Dave, David Sondak, Will Claybaugh and Pavlos Protopaps


# Introduction
As a data scientist, I often find myself looking for external data sources that could be relevant for my machine learning projects. The problem is that it is uncommon to find open source data sets that perfectly correspond to what you are looking for, or free APIs that give you access to data. In this case, web scraping can be one solution to get more data.



### How can we get to those web pages?
In order to answer those questions, we need to understand a little how websites work. Websites are created using HTML (Hypertext Markup Language), along with CSS (Cascading Style Sheets) and JavaScript. HTML elements are separated by tags and they directly introduce content to the web page. Here is what a basic HTML document looks like:

In [1]:
from IPython.display import HTML
import numpy as np
import urllib.request
import bs4 #this is beautiful soup
import time
#import operator
#import socket
#import cPickle
import re # regular expressions
from bs4 import BeautifulSoup
import requests
from pandas import Series
import pandas as pd
from pandas import DataFrame

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set_context("talk")
sns.set_style("white")

Scraping with Python:
=====================

* scraping is all about HTML tags
* bad news: 
    - need to learn about tags
    - websites can be ugly

HTML
=====

* HyperText Markup Language

* standard for creating webpages

* HTML tags 
    - have angle brackets
    - typically come in pairs

This is an example for a minimal webpage defined in HTML tags. The root tag is `<html>` and then you have the `<head>` tag. This part of the page typically includes the title of the page and might also have other meta information like the author or keywords that are important for search engines. The `<body>` tag marks the actual content of the page. You can play around with the `<h2>` tag trying different header levels. They range from 1 to 6. 

In [2]:
htmlString = """<!DOCTYPE html>
<html>
  <head>
    <title>This is a title</title>
  </head>
  <body>
    <h6> Test </h6>
    <p>Hello world!</p>
  </body>
</html>"""

htmlOutput = HTML(htmlString)
htmlOutput

Useful Tags
===========

* heading
`<h1></h1> ... <h6></h6>`

* paragraph
`<p></p>` 

* line break
`<br>` 

* link with attribute

`<a href="http://www.example.com/">An example link</a>`


#  Warm-up: get the content of the main page
First let’s use the requests module to get the HTML of the website’s main page.


In [4]:
main_url = "http://books.toscrape.com/index.html"

import requests
result = requests.get(main_url)

result.text[:1000]


'<!DOCTYPE html>\n<!--[if lt IE 7]>      <html lang="en-us" class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->\n<!--[if IE 7]>         <html lang="en-us" class="no-js lt-ie9 lt-ie8"> <![endif]-->\n<!--[if IE 8]>         <html lang="en-us" class="no-js lt-ie9"> <![endif]-->\n<!--[if gt IE 8]><!--> <html lang="en-us" class="no-js"> <!--<![endif]-->\n    <head>\n        <title>\n    All products | Books to Scrape - Sandbox\n</title>\n\n        <meta http-equiv="content-type" content="text/html; charset=UTF-8" />\n        <meta name="created" content="24th Jun 2016 09:29" />\n        <meta name="description" content="" />\n        <meta name="viewport" content="width=device-width" />\n        <meta name="robots" content="NOARCHIVE,NOCACHE" />\n\n        <!-- Le HTML5 shim, for IE6-8 support of HTML elements -->\n        <!--[if lt IE 9]>\n        <script src="//html5shim.googlecode.com/svn/trunk/html5.js"></script>\n        <![endif]-->\n\n        \n            <link rel="shortcut icon" href

The result is quite messy! Let’s make this more readable:

# Beautiful Soup


* designed to make your life easier
* many good functions for parsing html code

In [5]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(result.text, 'html.parser')
print(soup.prettify()[:1000])

<!DOCTYPE html>
<!--[if lt IE 7]>      <html lang="en-us" class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->
<!--[if IE 7]>         <html lang="en-us" class="no-js lt-ie9 lt-ie8"> <![endif]-->
<!--[if IE 8]>         <html lang="en-us" class="no-js lt-ie9"> <![endif]-->
<!--[if gt IE 8]><!-->
<html class="no-js" lang="en-us">
 <!--<![endif]-->
 <head>
  <title>
   All products | Books to Scrape - Sandbox
  </title>
  <meta content="text/html; charset=utf-8" http-equiv="content-type"/>
  <meta content="24th Jun 2016 09:29" name="created"/>
  <meta content="" name="description"/>
  <meta content="width=device-width" name="viewport"/>
  <meta content="NOARCHIVE,NOCACHE" name="robots"/>
  <!-- Le HTML5 shim, for IE6-8 support of HTML elements -->
  <!--[if lt IE 9]>
        <script src="//html5shim.googlecode.com/svn/trunk/html5.js"></script>
        <![endif]-->
  <link href="static/oscar/favicon.ico" rel="shortcut icon"/>
  <link href="static/oscar/css/styles.css" rel="stylesheet" type="tex

## Find book URLs on the main page
Now let’s start to dive deeper into the subject. In order to get the book data, we need to be able to access their product page. The first step consist in finding the URL of every book product page.
In your browser, go onto the website main page, right-click on the name of a product and click on inspect. This will show you the HTML part of the web page corresponding to this element. Congratulations, you have found the first book link!

Note the structure of the HTML code:

![avatar](12.png)

BeautifulSoup enables us to find those special ‘article’ tags. We can wall the find() function in order to find the first occurence of this tag in the HTML:





In [8]:
res=soup.find("article", class_ = "product_pod")

We still have too much information.
Let's dive deeper in the tree by adding the other child tags:


In [6]:
soup.find("article", class_ = "product_pod").div.a['href']

'catalogue/a-light-in-the-attic_1000/index.html'

## Now let’s gather all the products URLs on the main web page at once using the findAll() function:

In [7]:
main_page_products_urls = [x.div.a['href'] for x in soup.findAll("article", class_ = "product_pod")]

In [8]:
main_page_products_urls

['catalogue/a-light-in-the-attic_1000/index.html',
 'catalogue/tipping-the-velvet_999/index.html',
 'catalogue/soumission_998/index.html',
 'catalogue/sharp-objects_997/index.html',
 'catalogue/sapiens-a-brief-history-of-humankind_996/index.html',
 'catalogue/the-requiem-red_995/index.html',
 'catalogue/the-dirty-little-secrets-of-getting-your-dream-job_994/index.html',
 'catalogue/the-coming-woman-a-novel-based-on-the-life-of-the-infamous-feminist-victoria-woodhull_993/index.html',
 'catalogue/the-boys-in-the-boat-nine-americans-and-their-epic-quest-for-gold-at-the-1936-berlin-olympics_992/index.html',
 'catalogue/the-black-maria_991/index.html',
 'catalogue/starving-hearts-triangular-trade-trilogy-1_990/index.html',
 'catalogue/shakespeares-sonnets_989/index.html',
 'catalogue/set-me-free_988/index.html',
 'catalogue/scott-pilgrims-precious-little-life-scott-pilgrim-1_987/index.html',
 'catalogue/rip-it-up-and-start-again_986/index.html',
 'catalogue/our-band-could-be-your-life-scene

This function is very handy for finding all the values at once, but you have to check that all the information collected is relevant. Sometimes one same tag can contain completely different data. 

That is why it is important to be as specific as possible when choosing the tags.

Here we decided to rely on the tag ‘article’ with the ‘product_pod’ class because this seems to be a very specific tag and it is unlikely that we can find data other than product data in it.

The following cell just defines a url as a string and then reads the data from that url using the `urllib` library. If you uncomment the print command you see that we got the whole HTML content of the page into the string variable source.

## More detials


In [10]:
main_url = "http://books.toscrape.com/index.html"
result = requests.get(main_url)
soup = BeautifulSoup(result.text, 'html.parser')
first_tag = soup.find('a')
## get attribute `href`
first_tag.get('href')
## get all links in the page
link_list = [l.get('href') for l in soup.findAll('a')]
link_list

['index.html',
 'index.html',
 'catalogue/category/books_1/index.html',
 'catalogue/category/books/travel_2/index.html',
 'catalogue/category/books/mystery_3/index.html',
 'catalogue/category/books/historical-fiction_4/index.html',
 'catalogue/category/books/sequential-art_5/index.html',
 'catalogue/category/books/classics_6/index.html',
 'catalogue/category/books/philosophy_7/index.html',
 'catalogue/category/books/romance_8/index.html',
 'catalogue/category/books/womens-fiction_9/index.html',
 'catalogue/category/books/fiction_10/index.html',
 'catalogue/category/books/childrens_11/index.html',
 'catalogue/category/books/religion_12/index.html',
 'catalogue/category/books/nonfiction_13/index.html',
 'catalogue/category/books/music_14/index.html',
 'catalogue/category/books/default_15/index.html',
 'catalogue/category/books/science-fiction_16/index.html',
 'catalogue/category/books/sports-and-games_17/index.html',
 'catalogue/category/books/add-a-comment_18/index.html',
 'catalogue/ca

In [11]:
# create an empty list to collect the valid links
external_links = []

# write a loop to filter the links
# if it starts with 'http' we are happy
for l in link_list: #main_page_products_urls
    if l[:4] == 'http':
        l.startswith('http')
        external_links.append(l)

# this throws an error! It says something about 'NoneType'

In [12]:
# Let's filter those objects out in the for loop
external_links = []

# write a loop to filter the links
# if it is not None and starts with 'http' we are happy
for l in link_list:
    if l is not None and l[:4] == 'http':
        l.startswith('http')
        external_links.append(l)
        


Note: The above `if` condition works because of lazy evaluation in Python. The `and` statement becomes `False` if the first part is `False`, so there is no need to ever evaluate the second part. Thus a `None` entry in the list gets never asked about its first four characters. 

In [13]:
# and we can put this in a list comprehension as well, it almost reads like 
# a sentence.

res=[l for l in link_list if l is not None and l.startswith('http')]

# Python Tips

`tmp` now is actually a list of lists containing the hall of fame entries. 
Here is some advanced Python on how to print really just one entry per list item.

The cool things about this are: 
* The use of `""` to just access the `join` function of strings.
* The `join` function itself
* that you can actually have two nested for loops in a list comprehension

In [21]:
## reformat into a list containing strings
exp1=["a","b","c"]
exp2 = []
for li in exp1:
    exp2.append(li+"s")
exp2    

['as', 'bs', 'cs']

In [22]:
"ss".join("test")


'tssessssst'

#  Sum up the above as a function 



In [14]:
def getAndParseURL(url):
    result = requests.get(url)
    soup = BeautifulSoup(result.text, 'html.parser')
    return(soup)

def getBooksURLs(url):
    soup = getAndParseURL(url)
    # remove the index.html part of the base url before returning the results
    return(["http://books.toscrape.com/" + x.a.get('href') for x in soup.findAll( "div", class_="image_container")])



# Let’s do it!

In [15]:
url = "http://books.toscrape.com/index.html"
booksURLs=getBooksURLs(url)

In [16]:
booksURLs

['http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html',
 'http://books.toscrape.com/catalogue/tipping-the-velvet_999/index.html',
 'http://books.toscrape.com/catalogue/soumission_998/index.html',
 'http://books.toscrape.com/catalogue/sharp-objects_997/index.html',
 'http://books.toscrape.com/catalogue/sapiens-a-brief-history-of-humankind_996/index.html',
 'http://books.toscrape.com/catalogue/the-requiem-red_995/index.html',
 'http://books.toscrape.com/catalogue/the-dirty-little-secrets-of-getting-your-dream-job_994/index.html',
 'http://books.toscrape.com/catalogue/the-coming-woman-a-novel-based-on-the-life-of-the-infamous-feminist-victoria-woodhull_993/index.html',
 'http://books.toscrape.com/catalogue/the-boys-in-the-boat-nine-americans-and-their-epic-quest-for-gold-at-the-1936-berlin-olympics_992/index.html',
 'http://books.toscrape.com/catalogue/the-black-maria_991/index.html',
 'http://books.toscrape.com/catalogue/starving-hearts-triangular-trade-trilogy-1_990

In [17]:
soup.findAll("p")[-1].text

'\n\n    \n        In stock\n    \n'

In [19]:
names = []
prices = []
nb_in_stock = []
img_urls = []
categories = []
ratings = []

# scrape data for every book URL: this may take some time
for url in booksURLs:
    time.sleep(1)
    soup = getAndParseURL(url)
    # product name

    # product price
    prices.append(soup.find("p", class_ = "price_color").text[2:]) # get rid of the pound sign
    # number of available products
    nb_in_stock.append(re.sub("[^0-9]", "", soup.find("p", class_ = "instock availability").text)) # get rid of non numerical characters
    # image url
    img_urls.append(url.replace("index.html", "") + soup.find("img").get("src"))
    # product category
    categories.append(soup.find("a", href = re.compile("../category/books/")).get("href").split("/")[3])
    # ratings

    
# add data into pandas df
import pandas as pd

scraped_data = pd.DataFrame({'price': prices, 'nb_in_stock': nb_in_stock, "url_img": img_urls, "product_category": categories})


In [20]:
scraped_data.head(10)

,price,nb_in_stock,url_img,product_category
0,51.77,22,http://books.toscrape.com/catalogue/a-light-in...,poetry_23
1,53.74,20,http://books.toscrape.com/catalogue/tipping-th...,historical-fiction_4
2,50.10,20,http://books.toscrape.com/catalogue/soumission...,fiction_10
3,47.82,20,http://books.toscrape.com/catalogue/sharp-obje...,mystery_3
4,54.23,20,http://books.toscrape.com/catalogue/sapiens-a-...,history_32
5,22.65,19,http://books.toscrape.com/catalogue/the-requie...,young-adult_21
6,33.34,19,http://books.toscrape.com/catalogue/the-dirty-...,business_35
7,17.93,19,http://books.toscrape.com/catalogue/the-coming...,default_15
8,22.60,19,http://books.toscrape.com/catalogue/the-boys-i...,default_15
9,52.15,19,http://books.toscrape.com/catalogue/the-black-...,poetry_23
